In [17]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [18]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [21]:
df = pd.DataFrame(list(analyseCol.find({'soort': 'Stelling'})))
df['herkomst'] = np.NaN
dct = df.to_dict()
#simplejson.loads(simplejson.dumps(df.to_dict(), ignore_nan=True))
#dict(filter(lambda k: not isnan(df.to_dict()[k]), df.to_dict()))

[v.dropna().to_dict() for k,v in df.iterrows()]

[{'_id': ObjectId('6200424b0925906af160d32a'),
  'brondata': {'_id': ObjectId('6200424b0925906af160d32a'),
   'stelling': 'A',
   'inhoud': 'AARDEWERK',
   'table': 'stellingen',
   'project': 'MAGAZIJN',
   'bron': 'opgravingMAGAZIJN',
   'loadtime': '2022-02-06T21:48:57+00:00'},
  'stelling': 'A',
  'inhoud': 'AARDEWERK',
  'table': 'stellingen',
  'soort': 'Stelling',
  'ID': 0,
  'key': 'SA'},
 {'_id': ObjectId('6200424cddf14a3212c4ad91'),
  'brondata': {'_id': ObjectId('6200424cddf14a3212c4ad91'),
   'stelling': 'A',
   'inhoud': 'AARDEWERK',
   'table': 'stellingen',
   'project': 'MAGAZIJN',
   'bron': 'opgravingMAGAZIJN',
   'loadtime': '2022-02-06T21:48:57+00:00'},
  'stelling': 'A',
  'inhoud': 'AARDEWERK',
  'table': 'stellingen',
  'soort': 'Stelling',
  'ID': 22,
  'key': 'SA'},
 {'_id': ObjectId('6200424b0925906af160d32b'),
  'brondata': {'_id': ObjectId('6200424b0925906af160d32b'),
   'stelling': 'B',
   'inhoud': 'AARDEWERK',
   'table': 'stellingen',
   'project': 'MAG